# Cross Validation, Model Selection & Regularisation

In this notebook we will introduce the concepts below, and how they can be implemented in `scikit-learn`.
* Cross validation - a method for estimating the test error rate when test data is not available
* Model selection - how we use cross validation to select which model (from a selection) we should use for a particular data set
* Regularisation - an adaptation of linear regression to make it more flexible

[This video](https://www.youtube.com/watch?v=DQWI1kvmwRg) describes some of the ideas you will face in the coming notebook. The ideas we are covering here are described much more throughly in **ISLR** (see suggested sections in module overview).

https://trevorhastie.github.io/ISLR/

In [1]:
import pandas as pd
import numpy as np

## Cross validation

We have already seen the notion of splitting *test* and *training sets* in order to asses model performance. Now we will introduce the idea of *validation sets*.

Once again **ISTL** (Section 5.1) provides a very clear overview of how these technqiues work:

>Resampling methods are an indispensable tool in modern statistics. They
involve repeatedly drawing samples from a training set and refitting a model
of interest on each sample in order to obtain additional information about
the fitted model. For example, in order to estimate the variability of a linear
regression fit, we can repeatedly draw different samples from the training
data, fit a linear regression to each new sample, and then examine the
extent to which the resulting fits differ. Such an approach may allow us to
obtain information that would not be available from fitting the model only
once using the original training sample.

>Resampling approaches can be computationally expensive, because they
involve fitting the same statistical method multiple times using different
subsets of the training data. However, due to recent advances in computing
power, the computational requirements of resampling methods generally
are not prohibitive. [...] cross-validation can be used to estimate the test
error associated with a given statistical learning method in order to evaluate
its performance, or to select the appropriate level of flexibility. The process
of evaluating a model’s performance is known as model assessment, whereas model
the process of selecting the proper level of flexibility for a model is known as assessment
model selection.

The [diagram](https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7) below illustrates this process. Here the train and validation sets are used to do model assesment and model selection (NOTE: the test set is strictly forbidden from being used in any way during this process!). Once a model is selected the test set is used to do a final assesment of performance to see if the model selected will generalise as well as predicted.

<img src="../images/testtrainvalid.png" width="450px">

There is a simple overview these ideas [here](https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7), and a more thorough overview [here](https://machinelearningmastery.com/difference-test-validation-datasets/).

### K-fold cross validation

A very common method called *k-fold* is often used, which actually splits the training set multiple times. This allows us to assess the accuracy of the model over $k$ validation splits of data. The [image below](http://ethen8181.github.io/machine-learning/model_selection/model_selection.html) illustrates how this works for $k = 5$ splits.

<img src="../images/kfold.png" width="450px">

The `scikit-learn` documentation offers a simple [example](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) of implementing k-fold on some dummy data. We will examine this below. NOTE: The `scikit-learn` documentation is **FANSTASTIC(!)** and contains working examples of every function, it should always be the first place you look when you wish to implement a new function.

**Task 1:** 

* Have a look at the code below and check you understand what is going on. (Add some print statements in various places to help.)

In [8]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([['A', 'B'], ['C', 'D'], ['E', 'F'], ['G', 'H']])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=3) # here we choose the number of folds (or splits) we will make
kf.get_n_splits(X)

for train_index, valid_index in kf.split(X): # kf.split(X) is an iterable which gives us the indices of the data in each fold
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

In [19]:
for train_index, valid_index in kf.split(X):
    print(train_index, valid_index)
    X_train, X_valid = X[train_index], X[valid_index]
    print(f"X_train: {X_train}")
    print(f"X_valid: {X_valid}")
    y_train, y_valid = y[train_index], y[valid_index]
    print(f"y_train; {y_train}")
    print(f"y_valid: {y_valid}")
    print("#####")

[2 3] [0 1]
X_train: [['E' 'F']
 ['G' 'H']]
X_valid: [['A' 'B']
 ['C' 'D']]
y_train; [3 4]
y_valid: [1 2]
#####
[0 1 3] [2]
X_train: [['A' 'B']
 ['C' 'D']
 ['G' 'H']]
X_valid: [['E' 'F']]
y_train; [1 2 4]
y_valid: [3]
#####
[0 1 2] [3]
X_train: [['A' 'B']
 ['C' 'D']
 ['E' 'F']]
X_valid: [['G' 'H']]
y_train; [1 2 3]
y_valid: [4]
#####


We can apply this to any data set to perform operations as it gives us the dataframe/numpy array indicies in each loop to select the appropriate data for each fold. For example we can apply this to the auto data set. This contains rows which correspond to individual cars and their attributes.

In [48]:
auto = pd.read_csv('../data/Auto.csv')
auto = auto[auto.horsepower != '?']
auto['horsepower'] = auto.horsepower.astype(int)
auto.reset_index(inplace=True, drop=True)
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [49]:
mpg = auto.pop('mpg') # mpg will be our target and so we remove this into a seperate array

Say we are trying to predict 'mpg' from our other automobile data features. We can use KFold to iterate over the number of splits we choose.

**Task 2:**
* try adding print statements for the size of the dataframes in each split
* try increasing the number of splits and re-run your code
* use the code below to print a car name contained in the train and validation data set, for each split



In [50]:
kf = KFold(n_splits=2) # here we choose the number of folds (or splits) we will make
kf.get_n_splits(auto)

print(kf)


split_counter = 1
for train_index, valid_index in kf.split(auto): # kf.split(X) is an iterable which gives us the indices of the data in each fold
    print('-'*60)
    print('This is split no: {}'.format(split_counter))
    split_counter += 1 
    X_train, X_valid = auto.iloc[train_index], auto.iloc[valid_index] # must use .iloc because its a dataframe this time
    y_train, y_valid = mpg[train_index], mpg[valid_index]
    
    # your code here
    print('Train car name here')
    print('Validation car name here')

KFold(n_splits=2, random_state=None, shuffle=False)
------------------------------------------------------------
This is split no: 1
Train car name here
Validation car name here
------------------------------------------------------------
This is split no: 2
Train car name here
Validation car name here


In [52]:
#### Your solution here

auto_c = auto.copy()

kf = KFold(n_splits=9) # here we choose the number of folds (or splits) we will make
kf.get_n_splits(auto_c)

print(kf)

split_counter = 1
for train_index, valid_index in kf.split(auto_c): # kf.split(X) is an iterable which gives us the indices of the data in each fold
    print('-'*60)
    print('This is split no: {}'.format(split_counter))
    split_counter += 1 
    X_train, X_valid = auto_c.iloc[train_index], auto_c.iloc[valid_index] # must use .iloc because its a dataframe this time
    y_train, y_valid = mpg[train_index], mpg[valid_index]
    
    # your code here
    print(f"Dataset size: training is {len(X_train)} and validation is {len(X_valid)}")
    car = X_train.sample()
    car_valid = X_valid.sample()
    print(f"One random car from the dataset is: {car.name}")
    print(f"One random car from the validation dataset is: {car_valid.name}")

KFold(n_splits=9, random_state=None, shuffle=False)
------------------------------------------------------------
This is split no: 1
Dataset size: training is 348 and validation is 44
One random car from the dataset is: 180    fiat 131
Name: name, dtype: object
One random car from the validation dataset is: 32    amc gremlin
Name: name, dtype: object
------------------------------------------------------------
This is split no: 2
Dataset size: training is 348 and validation is 44
One random car from the dataset is: 364    chevrolet cavalier 2-door
Name: name, dtype: object
One random car from the validation dataset is: 72    chevrolet chevelle concours (sw)
Name: name, dtype: object
------------------------------------------------------------
This is split no: 3
Dataset size: training is 348 and validation is 44
One random car from the dataset is: 86    chevrolet malibu
Name: name, dtype: object
One random car from the validation dataset is: 107    toyota carina
Name: name, dtype: obje

In [53]:
auto = auto.drop('name', axis=1, errors='ignore') # we do not need the car names so remove for now.

In [56]:
auto

,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,8,307.0,130,3504,12.0,70,1
1,8,350.0,165,3693,11.5,70,1
2,8,318.0,150,3436,11.0,70,1
3,8,304.0,150,3433,12.0,70,1
4,8,302.0,140,3449,10.5,70,1
...,...,...,...,...,...,...,...
387,4,140.0,86,2790,15.6,82,1
388,4,97.0,52,2130,24.6,82,2
389,4,135.0,84,2295,11.6,82,1
390,4,120.0,79,2625,18.6,82,1


We can use the same loop to fit and evaluate our linear regression model on each train/validation split:

In [54]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


split_counter = 1
mse_scores = [] # create empty list to append mse scores for each split
kf = KFold(n_splits=5)
kf.get_n_splits(auto)

for train_index, valid_index in kf.split(auto): # kf.split(X) is an iterable which gives us the indices of the data in each fold
    print('-'*60)
    print('This is split no: {}'.format(split_counter))
    print('Model results')
    split_counter += 1 
    X_train, X_valid = auto.iloc[train_index], auto.iloc[valid_index] # must use .iloc because its a dataframe this time
    y_train, y_valid = mpg[train_index], mpg[valid_index]
    
    
    #### fit polynomial to train data in this split
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    
    #### eval & print MSE training results in this split
    mpg_train_pred = lin_reg.predict(X_train)
    mse_train = mean_squared_error(y_train, mpg_train_pred)
    print('training MSE: {0}'.format(mse_train))
    
    #### do the same for validation split
    mpg_valid_pred = lin_reg.predict(X_valid)
    mse_valid = mean_squared_error(y_valid, mpg_valid_pred)
    print('validation MSE: {0}'.format(mse_valid))
    
    mse_scores.append(mse_valid) # assign validation MSE score to list

------------------------------------------------------------
This is split no: 1
Model results
training MSE: 11.284070590566001
validation MSE: 14.974307651304196
------------------------------------------------------------
This is split no: 2
Model results
training MSE: 11.155158050598772
validation MSE: 10.905952427081138
------------------------------------------------------------
This is split no: 3
Model results
training MSE: 12.160105136871524
validation MSE: 5.991708610108158
------------------------------------------------------------
This is split no: 4
Model results
training MSE: 9.921674145405683
validation MSE: 15.587544657621601
------------------------------------------------------------
This is split no: 5
Model results
training MSE: 7.977511689294957
validation MSE: 27.844743081984205


When using k-fold cross validation we can analyse the validation MSE result for each split to assess the overall performance.

In [55]:
mse_scores  = np.array(mse_scores)
print('VALIDATION SET MSE SCORES')
print('mean MSE:', mse_scores.mean())
print('std MSE:', mse_scores.std())

VALIDATION SET MSE SCORES
mean MSE: 15.060851285619862
std MSE: 7.255691814890937


#### Cross validation in practice in sklearn
Most of the time we do not care about having access to each split. `Scikit-Learn` provide a much easier way to do all of this with the function `cross_val_score`. This allows us to do the same as above but in much less code.

In [57]:
from sklearn.model_selection import cross_val_score
lin_reg = LinearRegression()

cv_scores = cross_val_score(lin_reg, auto, mpg, cv = 5, scoring='neg_mean_squared_error') 

print('mean MSE:',np.mean(-cv_scores))
print('std MSE:',np.std(-cv_scores))

mean MSE: 15.060851285619862
std MSE: 7.255691814890937


In [58]:
cv_scores

array([-14.97430765, -10.90595243,  -5.99170861, -15.58754466,
       -27.84474308])

**Task 3:**

* Make sure you understand the output of the cross_val_score above (i.e. What is cv_scores?)
* Why is the scoring defined as negative MSE? Do some research
* Investigate what the `cross_val_predict` function does.
* Import and implement `cross_val_predict`on the same data as above.
* What are the outputs of this function?
* Can you use these to evaluate the results of your cross validation?
* Do the cross-validation scores give you confidence this model is providing a useful prediction?

# solution

* Make sure you understand the output of the cross_val_score above (i.e. What is cv_scores?)

"cv_scores" are the scores (in this case, the MSE) of the fit in each fold during each split of the data. 

* Why is the scoring defined as negative MSE? Do some research

SKlearn works by maximizing the cost function, so a higher value is better than a lower value. In this case, the MSE is negated to work with the API: the higher, the better; by definition, MSE can never be a negative number, but by flipping the sign we get the definition of the MSE follows the sklearn's API. Is just a convention.

---------------------------------------------------------

* Investigate what the `cross_val_predict` function does.
* Import and implement `cross_val_predict`on the same data as above.
* What are the outputs of this function?
* Can you use these to evaluate the results of your cross validation?
* Do the cross-validation scores give you confidence this model is providing a useful prediction?

In [61]:
from sklearn.model_selection import cross_val_predict

lin_reg = LinearRegression()

y_cross_val_predict = cross_val_predict(lin_reg, auto, mpg, cv = 5)
print("predicted labels by cross_val_predict:", y_cross_val_predict)

predicted labels by cross_val_predict: [13.18110394 11.64748588 13.09596676 12.93021809 12.98800698  7.88990469
  7.49736748  7.61390615  6.99125383 10.3725613  12.81242748 11.85752931
 12.48888573 15.42940075 22.40257956 17.5763258  17.90310929 19.52652471
 23.80542555 26.13155768 19.54330883 20.66744146 21.0440294  21.26752478
 18.88026307  4.63160711  5.68967386  5.4174852   3.90210194 24.73492723
 21.98111525 24.15704374 20.04217701 14.8816131  16.2121433  16.76693353
 16.12703348  9.36649493  8.35186884 10.24056977 10.03126363  4.77300228
  6.77627086  4.13308382 18.1920385  21.93936568 16.46707887 17.71163076
 21.91937357 23.6620962  24.35965545 23.98329061 27.59701624 28.41367685
 26.49486253 24.04690947 24.89520502 23.4135485  25.34117002 22.33315635
 23.0173159   9.88637782  9.8204652  10.75624436 11.28636682 13.24183437
  7.67074101  8.89384621  9.02346323  9.1477468  23.93588756 11.93793298
 11.46182309  9.97571971 11.16300067 18.91860546 23.02065154 19.82347731
 24.79715168

In [66]:
print("Cross Val Prediction score: {}".format(mean_squared_error(mpg,y_cross_val_predict)))

Cross Val Prediction score: 15.05003127926054


In [70]:
print(f"Percentual difference: {(np.mean(-cv_scores)-mean_squared_error(mpg,y_cross_val_predict))/(np.mean(-cv_scores))*100}")

Percentual difference: 0.07184193080541117


Yes, it does gives confidence: the mean of the validation scores using cross_val_score is very close, roughly the same, as the mean of the error using the predictions provided by cross_val_predict

## Excercise 1: wine cross-validation

You must predict the alcohol content of various wines based on their other attributes.

* Split the data into train and test data sets (Ensure you use the option: `random_state = 28`).
* Perform linear regression using k-fold cross validation(ensure you use 5 folds). Return the cross validation MSE errors. Return the mean and standard deviations of these.
* Evaluate the performance of the model on the test set.
* Compare the cross-validation error and the test error (MSE). What do you find? 
* Try removing the random_state option. What happens to your results? Explain why.

In [80]:
#### your solution here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

wine_data = pd.read_csv('../data/wine.csv')
X = wine_data.loc[:, wine_data.columns != "Alcohol"]
y = wine_data["Alcohol"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=28)

lin_reg = LinearRegression()

cv_scores = cross_val_score(lin_reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

In [81]:
cv_scores

array([-0.26271057, -0.38623508, -0.55062417, -0.65506213, -0.53110424])

In [82]:
print('mean MSE:',np.mean(-cv_scores))
print('std MSE:',np.std(-cv_scores))

mean MSE: 0.4771472419568702
std MSE: 0.13726764524713422


In [86]:
print("RMSE for cross val: {0}".format(np.sqrt(np.mean(-cv_scores))))

RMSE for cross val: 0.6907584541334765


In [84]:
lin_reg.fit(X_train, y_train)

y_pred_test = lin_reg.predict(X_test)

mse_test = mean_squared_error(y_test, y_pred_test)
print('test MSE: {0}'.format(mse_test))

test MSE: 0.477228311769869


In [85]:
rmse_test = np.sqrt(mse_test)
print("RMSE: {0}".format(rmse_test))

RMSE: 0.6908171333789204


They are quite close to each other

#### removing random state

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

lin_reg = LinearRegression()

cv_scores = cross_val_score(lin_reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

print('mean MSE:',np.mean(-cv_scores))
print('std MSE:',np.std(-cv_scores))

mean MSE: 0.39388394345635847
std MSE: 0.05374864366296432


It chqaged because it depends on the rqndom number generator in scikit learn's algorithm for splitting.

## Regularisation

An alternative to choosing models which contains smaller numbers of features is to use a method that *constrains* or *regularises* the coefficent estimates assigned to each feature, or that shrinks the coefficient towards zero. This technique is very similar to *least squares* which we have been using until now. Please refer to Section in 6.2 **ISTL** for a fuller explanation of this.

When we move to use a regularised linear regression for prediction the additional term means that we now have a model parameter that requires setting or tuning. These terms are referred to as *hyperparameters* in machine learning. In practice this introduces another additional unknown parameter which we must choose somewhere in our modelling. It is common practice to run several models, each with different values of this hyperparameter, and then assess the error of each using cross validation for comparison.

**Lasso vs Ridge** (just for information curiosity, no need to go in depth or spend to much time to understand the maths)

For now we will focus on how to implement Lasso and Ridge regression in sklearn. These are both types of regularised linear regression.

Lasso: objectif is to minimize
$$ RSS + \lambda \sum |\beta | $$
* can force coefficients exactly to zero: behaves thus as variable selection


Ridge: objectif is to minimize
$$ RSS + \lambda \sum \beta_i^2 $$
* does not force coefficients exactly to zero
* interestig when there are more predictors than observations

see further reading ILS, search for Ridge and Lasso

#### Lasso regression in sklearn

In this example we aim to predict credit rating of individual customers. To train and predict using a Lasso regression we follow much the same procedure as we have seen before in `scikit-learn`.

Additional sources for further reading :

https://www.datacamp.com/community/tutorials/tutorial-ridge-lasso-elastic-net

http://eric.univ-lyon2.fr/~ricco/cours/slides/regularized_regression.pdf

In [90]:
credit = pd.read_csv('../data/credit_modified.csv')
rating = credit.pop('Rating')
credit.head()

,Income,Limit,Cards,Age,Education,Gender,Student,Married,Balance,African American,Asian,Caucasian
0,14.891,3606,2,34,11,0,0,1,333,0,0,1
1,106.025,6645,3,82,15,1,1,1,903,0,1,0
2,104.593,7075,4,71,11,0,0,0,580,0,1,0
3,148.924,9504,3,36,11,1,0,0,964,0,1,0
4,55.882,4897,2,68,16,0,0,1,331,0,0,1


In [91]:
#### splitting train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(credit, rating, random_state = 91)

In [92]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha = 10)
cv_scores = cross_val_score(lasso, X_train, y_train, cv = 5, scoring='neg_mean_squared_error')
print('mean MSE:',np.mean(-cv_scores))
print('std MSE:',np.std(-cv_scores))

mean MSE: 148.3658024815141
std MSE: 22.577610846122713


Here we can alter the alpha parameter to change the amount of regularisation the model uses (try this yourself! - vary the value by at least factors of 10). With increases in regularisation we expect a reduction in the *variance* of the model.

In [94]:
alphas = [10, 20, 30, 40, 50]

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    cv_scores = cross_val_score(lasso, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print(f"mean MSE for alpha = {alpha} is :",np.mean(-cv_scores))
    print(f"std MSE for alpga = {alpha} is: ",np.std(-cv_scores))
    print("#"*20)

mean MSE for alpha = 10 is : 148.3658024815141
std MSE for alpga = 10 is:  22.577610846122713
####################
mean MSE for alpha = 20 is : 149.70532442006868
std MSE for alpga = 20 is:  19.056064042393746
####################
mean MSE for alpha = 30 is : 150.9130306232697
std MSE for alpga = 30 is:  17.2469045258343
####################
mean MSE for alpha = 40 is : 151.4361789478293
std MSE for alpga = 40 is:  17.375241368886083
####################
mean MSE for alpha = 50 is : 151.51784877468032
std MSE for alpga = 50 is:  17.419502103860072
####################


#### Using Lasso with grid search

In practice we do not want to vary hyperparameters by hand to find which value is best (the model with minimum cross validation error). Of course `scikit-learn` has a function that automates this for you. Using `GridSearchCV` we pass a dictionary of parameter values we wish to investigate. The function will fit each model we have listed and calculate the cross validation error of each. It provides all the results through the object it returns.

In [95]:
from sklearn.model_selection import GridSearchCV
lasso = Lasso(max_iter=10000)

param_grid = [
 {'alpha': [0.001, 0.01, 0.1, 1, 3, 10, 100, 1000]}
 ]

grid_search = GridSearchCV(lasso, param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

grid_results = pd.DataFrame(grid_search.cv_results_)

grid_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008900,0.000700,0.003093,0.000123,0.001,{'alpha': 0.001},-134.693531,-105.857049,-101.376944,-87.569097,-78.913304,-76.137420,-109.786540,-103.885686,-106.104721,-113.455931,-101.778022,16.450021,3
1,0.007852,0.000142,0.002865,0.000062,0.01,{'alpha': 0.01},-134.553201,-105.948212,-101.096381,-87.881139,-78.937345,-76.585584,-109.682491,-103.106018,-106.112572,-113.688788,-101.759173,16.329243,2
2,0.006570,0.000398,0.002880,0.000141,0.1,{'alpha': 0.1},-133.132694,-106.809300,-98.722275,-91.394863,-79.351692,-78.012317,-108.680986,-96.604537,-106.093953,-115.062267,-101.386489,15.688554,1
3,0.005532,0.000052,0.002824,0.000130,1,{'alpha': 1},-127.851853,-111.017555,-92.827631,-106.006184,-89.048154,-82.339570,-105.158164,-94.934777,-102.310102,-122.139519,-103.363351,13.605513,4
4,0.005356,0.000026,0.002809,0.000112,3,{'alpha': 3},-132.499698,-117.173048,-106.238114,-112.301661,-94.847286,-76.537019,-102.509314,-98.851062,-105.902336,-131.338637,-107.819817,15.910751,5


`GridSearchCV` also returns a model (with the best hyperparmeter combination it found) which has been fitted one final time to all of the training data. Therefore it is ready to make predictions on the testing set. The model can be accessed like this:

In [97]:
grid_search.best_estimator_

Lasso(alpha=0.1, max_iter=10000)

In [120]:
grid_results.loc[grid_results["param_alpha"] == 0.1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
2,0.00657,0.000398,0.00288,0.000141,0.1,{'alpha': 0.1},-133.132694,-106.8093,-98.722275,-91.394863,-79.351692,-78.012317,-108.680986,-96.604537,-106.093953,-115.062267,-101.386489,15.688554,1


**Task 4:**
* How many times will the lasso model be fitted when the GridSearchCV function is called above?

<details><summary>Hint</summary><br>
Check what the `refit=True` parameter does in GridSearchCV
</details>

* Look through the columns of the `grid_results` dataframe. Try and understand what the table contains.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

### ** It fits the models a number equal to the cv times all possible combinations of parameters in the "parameter grid". In this specific case, where there is only one hyperparameter with 8 possible values in the parameter grid, with cv=10 GridSearchCV will fit 10 times 8 = 80 models plus 1 last model with the best paramters found.

## Model selection

Congratulations! You have just done your first model hyperparameter tuning in `scikit-learn`! 

If we have a dataset for which we are interested in developing a predictive model. We do not know beforehand which model will perform best for this particular data or problem. Therefore, we fit and evaluate a number of different models to our data. The models could also be of varying type as well as flexibility (e.g. random forests, support vector machines, linear regression). We then need to decide which of our models we will choose to use in our final product.

As **ISLR** states:
> "we can directly estimate the test error using the validation set and cross-validation methods
discussed in Chapter 5. We can compute the validation set error or the
cross-validation error for each model under consideration, and then select
the model for which the resulting estimated test error is smallest."

This works as a simple rule, which we will follow for the remainder of this notebook. However in practice the selection can sometimes be a bit more nuanced. Read more detail [here](https://machinelearningmastery.com/a-gentle-introduction-to-model-selection-for-machine-learning/).

#### Task 5: Ridge regression in sklearn

Another type of regularised linear model is know as *Ridge regression*.

* Repeat the model prediction process above on the credit data but use a ridge regression model.
* Try replacing `GridSearchCV` with `RandomizedSearchCV`
* How do these functions differ?

In [116]:
#### Your solution here

credit = pd.read_csv('../data/credit_modified.csv')
rating = credit.pop('Rating')

#### splitting train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(credit, rating, random_state=91)

from sklearn.linear_model import Ridge

ridge = Ridge(alpha=10)
cv_scores = cross_val_score(ridge, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print('mean MSE:',np.mean(-cv_scores))
print('std MSE:',np.std(-cv_scores))

from sklearn.model_selection import RandomizedSearchCV

ridge = Ridge(max_iter=10000)

from scipy.stats import randint as sp_randInt

param_grid = [
 {'alpha': sp_randInt(0, 1000).rvs(1000)}
 ]

rand_search = RandomizedSearchCV(ridge, param_grid, cv=10, scoring='neg_mean_squared_error')
rand_search.fit(X_train, y_train)

rand_results = pd.DataFrame(rand_search.cv_results_)

rand_results.head()

mean MSE: 100.56334878030518
std MSE: 14.39228046971534


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004823,0.000203,0.002709,0.000212,946,{'alpha': 946},-158.492406,-129.490313,-137.958589,-126.804473,-103.833638,-67.877306,-105.062334,-111.670877,-122.877714,-153.602322,-121.766997,25.134344,10
1,0.004688,0.000027,0.002605,0.000075,494,{'alpha': 494},-148.031008,-120.741361,-122.673920,-117.496352,-94.626573,-67.098675,-101.314304,-104.638457,-115.587808,-142.235384,-113.444384,22.161492,5
2,0.004820,0.000253,0.002672,0.000164,498,{'alpha': 498},-148.152599,-120.840278,-122.854423,-117.607817,-94.727814,-67.093053,-101.344335,-104.719861,-115.670880,-142.374226,-113.538529,22.199845,6
3,0.004716,0.000043,0.002648,0.000111,588,{'alpha': 588},-150.730149,-122.947894,-126.657170,-119.941539,-96.901859,-67.064397,-102.064898,-106.442997,-117.439923,-145.268765,-115.545959,22.989553,7
4,0.004710,0.000032,0.002642,0.000108,344,{'alpha': 344},-143.023047,-116.682238,-115.109548,-112.725852,-90.541011,-67.728049,-100.451934,-101.250932,-112.191601,-136.262938,-109.596715,20.481485,3


In [117]:
#### change to randomised search CV
rand_search.best_estimator_

Ridge(alpha=255, max_iter=10000)

In [119]:
rand_results.loc[rand_results["param_alpha"] == 255]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
6,0.004669,0.00002,0.002573,0.000065,255,{'alpha': 255},-139.682,-113.940164,-109.841922,-109.188558,-87.867132,-68.71012,-100.421656,-98.901389,-109.947189,-131.844152,-107.034428,19.225231,1


## Excercise 2: Moneyball

Moneyball, as well as being a fantastic story, is also a true story of statistical methods being applied in a real world context to make predictions for decision making. [The film Moneyball](https://www.youtube.com/watch?v=-4QPVo0UIzc) is well worth a watch if you have time. As well as in baseball most major competitive sports teams are now using data science to improve their performance, e.g. [football](http://outsideoftheboot.com/2013/06/26/rise-of-data-analysis-in-football/),...

In this excercise you have been hired by Oakland Athletics general manager Billy Beane. Your first mission is to predict the salary each player will make based on other information that is available. This will allow Billy to understand what price he should pay for players in the next transfer season.

You must:
* Import and prepare the data
* Create a train and test set
* Implement a regularised model of your choice (Ridge or Lasso)
* Choose optimal parameters for your regularised model
* Estimate test-error using k-fold cross validation
* Calculate the true test-error
* Run a base line model to compare your model results. Base line model is the most simple approach based on strategy of choice (mean or other). It is then used as reference to conclude whether more complex models are better or not: see DummyRegressor in the sklearn library.

HINT 1
* Some values are missing. You can drop these rows.

HINT 2
* Some columns do not contain numerical values. You can drop these columns. In the Machine Learning model you'll learn more about labeling categorical data

In [141]:
#### Your solution here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randInt

# import the hitters.csv dataset

#### Your solution here
# import the hitters.csv dataset

# clean data

hitters_data = pd.read_csv("../data/Hitters.csv")
hitters_data = hitters_data.select_dtypes(include=np.number)
hitters_data.dropna(axis=0, inplace=True)
salary = hitters_data.pop("Salary")
hitters_data.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25


In [142]:
hitters_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 263 entries, 1 to 321
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   AtBat    263 non-null    int64
 1   Hits     263 non-null    int64
 2   HmRun    263 non-null    int64
 3   Runs     263 non-null    int64
 4   RBI      263 non-null    int64
 5   Walks    263 non-null    int64
 6   Years    263 non-null    int64
 7   CAtBat   263 non-null    int64
 8   CHits    263 non-null    int64
 9   CHmRun   263 non-null    int64
 10  CRuns    263 non-null    int64
 11  CRBI     263 non-null    int64
 12  CWalks   263 non-null    int64
 13  PutOuts  263 non-null    int64
 14  Assists  263 non-null    int64
 15  Errors   263 non-null    int64
dtypes: int64(16)
memory usage: 34.9 KB


make train test split

In [143]:
# make train test split

X_train, X_test, y_train, y_test = train_test_split(credit, rating, random_state=91)

linear regression model build using CV to estimate what our model performance will be on data it hasnt seen

In [144]:
##### using k-fold cross val score

lasso = Lasso(alpha=10)
cv_scores = cross_val_score(lasso, X_train, y_train, cv=8, scoring='neg_mean_squared_error')
print('mean MSE:',np.mean(-cv_scores))
print('std MSE:',np.std(-cv_scores))

mean MSE: 146.81494098960226
std MSE: 23.416268075981936


In [146]:
# Print the best hyperparameter for model performance....

param_grid = [
 {"alpha": sp_randInt(0, 1000).rvs(1000),
 "fit_intercept": [True, False],
 "warm_start": [True, False],
 "positive": [True, False]
 }
 ]

lasso = Lasso(max_iter=10000)

rand_search = RandomizedSearchCV(lasso, param_grid, cv=10, scoring='neg_mean_squared_error')
rand_search.fit(X_train, y_train)

rand_results = pd.DataFrame(rand_search.cv_results_)
rand_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_warm_start,param_positive,param_fit_intercept,param_alpha,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005297,0.000975,0.003281,0.000650,False,True,True,810,"{'warm_start': False, 'positive': True, 'fit_i...",-188.714682,...,-143.089478,-157.682408,-103.321880,-149.715099,-122.633290,-144.707290,-184.700478,-152.229751,25.934215,4
1,0.004744,0.000012,0.002773,0.000131,True,False,False,326,"{'warm_start': True, 'positive': False, 'fit_i...",-404.927147,...,-410.446107,-231.941134,-327.138646,-299.617756,-464.361076,-354.282012,-294.293762,-369.317152,87.658762,8
2,0.005224,0.000038,0.002765,0.000111,True,False,False,45,"{'warm_start': True, 'positive': False, 'fit_i...",-344.176498,...,-276.597096,-210.026304,-221.917183,-245.037512,-200.801769,-177.251209,-164.269413,-242.598229,60.538396,5
3,0.004676,0.000006,0.002753,0.000118,True,False,False,462,"{'warm_start': True, 'positive': False, 'fit_i...",-407.154368,...,-414.000573,-230.328798,-328.957864,-295.904106,-476.666042,-354.274176,-295.957191,-371.134278,91.019445,9
4,0.004934,0.000023,0.002753,0.000111,True,True,True,64,"{'warm_start': True, 'positive': True, 'fit_in...",-188.232367,...,-146.083311,-157.785074,-102.205670,-146.611091,-126.589917,-142.766950,-181.603558,-151.904283,25.650790,1


In [151]:
print('mean MSE:', -rand_search.best_score_)

mean MSE: 151.90428275870158


In [152]:
rand_search.best_estimator_

Lasso(alpha=64, max_iter=10000, positive=True, warm_start=True)

In [154]:
y_pred_test = rand_search.predict(X_test)

mse_test = mean_squared_error(y_test, y_pred_test)
print('test MSE: {0}'.format(mse_test))

rmse_test = np.sqrt(mse_test)
print("RMSE: {0}".format(rmse_test))

test MSE: 147.61929628309193
RMSE: 12.149868159082711


**One important thing** to realise is that when we run our `GridSearchCV` for our lasso model we are changing the flexibility of our model and estimating our overall test error. In essence we are estimating this curve below, which we saw in the text book!

<img src="../images/bias_var_tradeoff.png" width="300px">

We can plot our cv values for each of our model flexibilities (alpha values)....

In [31]:
# plot cv values against alpha parameters

**Task 6:**
* Ridge regression works best when the input variables are standardised. (See section 6.2 **ISLR** for more details.). Try standardising your data before running your model. Do you find different results?
* does this model outperform a simple linear regression model?
* Which variables are most important in the model?
* Make a pairplot or scatter matrix: pd.plotting.scatter_matrix

**These and other things to consider in this problem...**

* We must actually standardise our *X* variables by scaling each *X* by the standard deviation, if not Ridge will not penalise each variable in the same way. We will learn a robust an easy way to do this in the intro to ML module.
* Once our variables are scaled we can use the $\beta$ coefficients that our model finds to see which has a greater influence in our model. We can find as we see below.
* The reason our model does not perform so well is that most of our *X* variables do not have a very clear linear relationship with *y* (see below)! Using non-linear models could be of use in this problem...again we will cover these soon!

In [54]:
# Your solution here

#standardize data


In [32]:
# run model and compare with results from non-standardized X


In [33]:
#### solution plot scatter matrix
